In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the Libraries 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px 
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
from pandas import DataFrame
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Reading the Data

In [ ]:
df = pd.read_csv("../input/world-population-by-year/WorldPopulation.csv") 

# View the Data

In [ ]:
df.columns

# Variables:
- Year: 1951 to 2020
- Population: World Population
- ChangePerc: Yearly Change in Percentage
- NetChange: Total Yearly Change
- Density: Density in P/Km²
- Urban: Urban Population
- UrbanPerc: Urban Population Percentage

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

# Descriptive Statistics



In [ ]:
df.describe()

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
import missingno as msn
msn.matrix(df)

In [ ]:
df.isnull().sum()

# There are no null values here. 

In [ ]:
plt.scatter(df['Year'],df[ 'Population'])

In [ ]:
x = df['Year']
y = df['Density']
plt.plot(x,y)
plt.xlabel('Year')
plt.ylabel('Density')

In [ ]:
x = df['Year']
y = df['Urban']
plt.plot(x,y)
plt.xlabel('Year')
plt.ylabel('Urban')

- This is Linear but I think Simple Linear Regression couldn't be a good choice. This should be Polynomial Regression.

In [ ]:
x = df['Year']
y = df['UrbanPerc']
plt.plot(x,y)
plt.xlabel('Year')
plt.ylabel('UrbanPerc')

In [ ]:
plt.xlabel('Year')
plt.ylabel('Density')
plt.scatter(df['Year'],df['Density'])

In [ ]:
plt.xlabel('Year')
plt.ylabel('Urban')
plt.scatter(df['Year'],df['Urban'])

In [ ]:
fig=plt.figure(figsize=(15,10))
plt.bar(df['Year'],df.Population)
plt.bar(df['Year'],df.Urban)
plt.xlabel("Year")
plt.show()

# There is a increment in Population and urban population with respect to Years Increse.

In [ ]:
plt.figure(figsize=(20,15))
sns.barplot(df.Year[:50],df.Density)

# There is a positive growth in the density also. 

In [ ]:
plt.hist(df.Urban)
plt.show

# Not Same Growth rate in Urban Population with the Years.

In [ ]:
plt.hist(df.NetChange)
plt.show

# NetChange is high or low but it is also not regular, 

# Correlation 

In [ ]:
corr=df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr,annot=True, cmap='RdYlBu')

# Now we can see clearly through the Heatmap our positive and Negative Relationship 

In [ ]:
!pip install dabl

In [ ]:
# comparison of all other attributes with respect to Math Marks

import dabl

plt.rcParams['figure.figsize'] = (18, 6)
plt.style.use('fivethirtyeight')
dabl.plot(df, target_col = 'Population')

In [ ]:
# Skewness of the Data. 

plt.subplot(1, 3, 1)
sns.distplot(df['Urban'],color = 'magenta')

plt.subplot(1, 3, 2)
sns.distplot(df['NetChange'],color = 'magenta')

plt.subplot(1, 3, 3)
sns.distplot(df['ChangePerc'],color = 'magenta')

plt.suptitle('Checking for Skewness', fontsize = 18)
plt.show()

# Predictions

# 1. We want to predict the Population of the Upcoming Years. 

# Linear Regression 

In [ ]:
x= df.iloc[:, :-6].values     # for Year 
y= df.iloc[:, 1].values        # For population 

In [ ]:
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 1/3, random_state=0)  

In [ ]:
#Fitting the Simple Linear Regression model to the training dataset  
from sklearn.linear_model import LinearRegression  
regressor= LinearRegression()  
regressor.fit(x_train, y_train)  

In [ ]:
#Prediction of Test and Training set result  
y_pred= regressor.predict(x_test)  
x_pred= regressor.predict(x_train)  

In [ ]:
plt.scatter(x_train, y_train, color="green")   
plt.plot(x_train, x_pred, color="red")    
plt.title("Population with Years ")  
plt.xlabel("Year")  
plt.ylabel("Population")  
plt.show()   

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(x_train,x_pred) 

In [ ]:
#visualizing the Test set results  
plt.scatter(x_test, y_test, color="blue")   
plt.plot(x_train, x_pred, color="red")    
plt.title("Population with Years ")  
plt.xlabel("Year")  
plt.ylabel("Population")  
plt.show()  

In [ ]:
print(regressor.score(x_test, y_test))

# Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures  
poly_regs= PolynomialFeatures(degree= 6)  # For better accuracy we can use 3,4,5,6 also. 
x_poly= poly_regs.fit_transform(x)  
lin_reg_2 =LinearRegression()  
lin_reg_2.fit(x_poly, y)  

In [ ]:
#Visulaizing the result for Polynomial Regression  
plt.scatter(x,y,color="blue")  
plt.plot(x, lin_reg_2.predict(poly_regs.fit_transform(x)), color="red")  
plt.title("Population with Years ")  
plt.xlabel("Year")  
plt.ylabel("Population")   
plt.show()  

In [ ]:
print(lin_reg_2.score(x_poly, y))

# XGBOOST

In [ ]:
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators = 100, max_depth = 85, random_state = 52).fit(x_train, y_train)
pred = model.predict(x_test)
pred
model.score(x_test, y_test)

# We can find that Our Polynomial Regression with this degree have highest score such as 99.98 % 

# Time Series 

In [ ]:
df1 = df[['Year', 'Population']]

In [ ]:
df1

In [ ]:
plt.plot(df1['Year'],df1['Population'])
plt.xlabel("Year")
plt.ylabel("Population")
plt.show()

In [ ]:
# perform Dickey-Fuller test
from statsmodels.tsa.stattools import adfuller

print('Results of Dickey-Fuller Test')

dftest = adfuller(df1.Population, autolag='AIC') 

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p value','#Lags used', 'No:of observations found'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    
print(dfoutput)

# Auto Arima Forecast in Python


In [ ]:
pip install pmdarima

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

model = pm.auto_arima(df1['Population'], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
model.plot_diagnostics(figsize=(7,5))
plt.show()

In [ ]:
# Forecast

n_periods = 2400
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = np.arange(len(df1['Population']), len(df1['Population'])+n_periods)

# make series for plotting purpose
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

# Plot
plt.plot(df1['Population'])
plt.plot(fc_series, color='darkgreen')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Final Forecast")
plt.show()

# Thanks 